In [1]:
import os
import re
import gzip
# name of channel to be scanned
channel_prefix = "https://www.youtube.com/"
#channel = "@StronnyCuttles"
channel = "@ShibiCottonbum"
#channel = "@ImmyBisou"
#channel = "@IceySnowpaws"
#channel = "@MercyModiste"
#channel = "@AzuraDulait"

In [9]:
try:   os.mkdir(channel)
except:   pass
try:   os.mkdir(f"{channel}/downloaded")
except:   pass
try:   os.mkdir(f"{channel}/processed")
except:   pass
try:   os.mkdir(f"{channel}/transcripts")
except:   pass
# file containing keywords
keywords_file = "keywords.txt"

# file containing thematic searches
thematic_file = "thematic.txt"

# scan channel for videos
# live videos
os.system(f"yt-dlp --get-id --skip-download {channel_prefix+channel} > {channel}/videos.txt")
videos = []
with open(f"{channel}/videos.txt", "r") as f:
    for line in f:
        if line.startswith("Error"):
            continue
        videos.append(line.strip())

# look for filenames in {channel}/downloaded for already downloaded videos
# the file name contains the video id in the following format:
# 【Supermarket Simulator】Definitely not drunk on the job 🐐🍼【VAllure】 [i3rrDSjokF4].webm
# where [i3rrDSjokF4] is the video id
downloaded = []
for file in os.listdir(f"{channel}/downloaded"):
    if file.endswith(".webm"):
        m = re.search(r"\[(.*?)\]", file)
        if m:
            downloaded.append(m.group(1))
            
# do the same for transcripts
transcripts = []
for file in os.listdir(f"{channel}/transcripts"):
    if file.endswith(".srt"):
        m = re.search(r"\[([^\[\]]+)\](?!.*\[)", file)
        if m:
            transcripts.append(m.group(1))


# compare list of videos to videos already downloaded
to_download = list(set(videos) - set(downloaded) - set(transcripts))
for download in to_download:
    print(f"{download}")

XHTQNnq_9Ak
d-VmQRibZoI
YMSUUWZ5sUI
GMU1FedXvJU
G_PIICieI_Y
yYTQMWZVZgs
uCPlqZ2dbgQ
nflOxUBlafE
nskqLTOxUOA
NGZB9YZq20s
uqLbMyxFUzY
i2ACzDlI1ds
iOCjBHb4t50


In [10]:
newly_downloaded = []
for video in to_download:
    print(f"Downloading {video}")
    output = os.system(f"yt-dlp -f bestaudio -P {channel}/downloaded {channel_prefix}watch?v={video}")
    if output!=0:
        print(f"Error downloading {video}")

Error downloading iOCjBHb4t50


In [2]:
#collect file names in {channel}/downloaded
downloaded_files = os.listdir(f"{channel}/downloaded")
for file in downloaded_files:
    if file.endswith(".webm") or file.endswith(".m4a"):
        #result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --output_dir --language English --model large-v3 \"{channel}/transcripts/\"")
        result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --language English --output_dir \"{channel}/transcripts/\"")
        if result==0:
            #move file to {channel}/processed
            try:
                os.rename(f"{channel}/downloaded/{file}", f"{channel}/processed/{file}")
            except:
                print(f"Error moving {file} to {channel}/processed")

In [3]:
#remove .tsv, .txt, .vtt files, (keeping .srt and .json), and compress json files to .json.gz in {channel}/transcripts
transcript_files = os.listdir(f"{channel}/transcripts")
for file in transcript_files:
    if file.endswith(".tsv") or file.endswith(".txt") or file.endswith(".vtt"):
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
    if file.endswith(".json"):
        with open(f"{channel}/transcripts/{file}", "rb") as f:
            with gzip.open(f"{channel}/transcripts/{file}.gz", "wb") as f_out:
                f_out.writelines(f)
        #remove original json file
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
        